In [1]:
import pandas as pd
import numpy as np

findf = pd.read_csv("./Data/finalData/findf_AllSeries_MergerAdjusted.csv")
findf["RSSD9999"] = pd.to_datetime(findf["RSSD9999"], format="%Y-%m-%d")
findf = findf.sort_values(["RSSD9999", "RSSD9001"])
findf = findf.set_index(["RSSD9999", "RSSD9001"])

In [2]:
# # To look for particular columns
findf.columns.str.match(".*1350").sum()

1

In [3]:
# Convert Year-to-date income statement variables to quarterly
def deYTM(series):
    q1obs = series[series.index.get_level_values(0).month == 3]
    series = series.groupby(series.index.get_level_values(1)).diff()
    return pd.concat([series[~series.index.isin(q1obs.index)], q1obs]).sort_index()

In [4]:
# Create relevant variables
findf['IEA'] = findf[['RCON0071', 'RCON2122', 'RCON0390', 'RCON2146', 'RCON3545', 'RCON1754', 'RCON1772', 'RCON1350', 'RCONB987', 'RCONB989']].sum(axis=1, skipna=True)
findf['savingDep'] = findf[['RCON6810', 'RCON0352']].sum(axis=1, skipna=True)
findf['demandDep'] = findf['RCON2215']
findf['timeDep'] = findf[['RCON6648', 'RCON6645', 'RCON6646', 'RCON2604', 'RCONJ473', 'RCONJ474']].sum(axis=1, skipna=True)
findf['intBearDep'] = findf[['RCON6636']]

findf['matAsset'] = findf[['RCONA549', 'RCONA550', 'RCONA551', 'RCONA552', 'RCONA553', 'RCONA554', 'RCONA555', 'RCONA556', 'RCONA557', 'RCONA558', 'RCONA559', 'RCONA560', 'RCONA561', 'RCONA562', 'RCONA564', 'RCONA565', 'RCONA566', 'RCONA567', 'RCONA568', 'RCONA569', 'RCONA570', 'RCONA571', 'RCONA572', 'RCONA573', 'RCONA574', 'RCONA575']].sum(axis=1, skipna=True)
findf['matLiab'] = findf[['RCONA579', 'RCONA580', 'RCONA581', 'RCONA582', 'RCONA584', 'RCONA585', 'RCONA586', 'RCONA587', 'RCONHK07', 'RCONHK08', 'RCONHK09', 'RCONHK10', 'RCONHK12', 'RCONHK13', 'RCONHK14', 'RCONHK15', 'RCONF055', 'RCONF056', 'RCONF057', 'RCONF058',  'RCONF060', 'RCONF061', 'RCONF062', 'RCONF063', 'demandDep', 'savingDep']].sum(axis=1, skipna=True)


findf['othAssets'] = findf['RCON2170'].sub(findf['matAsset'], fill_value = 0)
findf['othLiab'] = findf['RCON2948'].sub(findf['matLiab'], fill_value = 0)

findf["totAssets"] = findf["RCON2170"]
findf["qtyAssets"] = findf["RCON3368"]
findf["totLiab"] = findf["RCON2948"]
findf["totEquity"] = findf["RCON3210"]
findf["totLoans"] = findf["RCON2122"]
findf["logAssets"] = np.log(findf["totAssets"])
findf["deltaLogA"] = findf.groupby(findf.index.get_level_values(1))["logAssets"].diff()
findf['deltaLogA'] = findf["deltaLogA"].replace([np.inf, -np.inf], np.nan)
findf["totTreasuries"] = findf[["RCONA549", "RCONA550", "RCONA551", "RCONA552", "RCONA553", "RCONA554"]].sum(axis = 1, skipna = True)

shiftedIEA = findf[["IEA"]].groupby(findf.index.get_level_values(1)).shift(1).IEA

findf['intExpenseRateAnn'] = deYTM(findf['RIAD4073'].sub(findf['RIADC900'], fill_value = 0))/shiftedIEA
findf['intIncomeRateAnn'] = deYTM(findf['RIAD4107'].sub(findf['RIADC899'], fill_value = 0).sub(findf['RIADB525'], fill_value = 0).sub(findf['RIAD4842'], fill_value = 0))/shiftedIEA
findf['niiRateAnn'] = deYTM(findf['RIAD4074'].sub(findf['RIADC899'], fill_value = 0).add(findf['RIADC900'], fill_value = 0).sub(findf['RIADB525'], fill_value = 0).sub(findf['RIAD4842'], fill_value = 0))/shiftedIEA
findf['nniRateAnn'] = deYTM(findf['RIAD4079'].sub(findf['RIAD4093'], fill_value = 0).sub(findf['RIADC902'], fill_value = 0).sub(findf['RIADC903'], fill_value = 0).sub(findf['RIADC904'], fill_value = 0).sub(findf['RIADC905'], fill_value = 0).add(findf['RIADC907'], fill_value = 0).sub(findf['RIAD4097'], fill_value = 0).add(findf['RIAD4239'], fill_value = 0))/shiftedIEA
findf['roaAnn'] = deYTM(findf['RIAD4340'].sub(findf['RIADC914'], fill_value = 0).sub(findf['RIAD4341'], fill_value = 0))/shiftedIEA
findf['intIncLoans'] = deYTM(findf['RIAD4010'])/shiftedIEA
findf['intIncFed'] = deYTM(findf["RIAD4020"])/shiftedIEA
findf['intIncOthSec'] = deYTM(findf["RIAD4060"])/shiftedIEA
findf['IntIncTrade'] = deYTM(findf["RIAD4069"])/shiftedIEA
findf['intIncDepInst'] = deYTM(findf["RIAD4115"])/shiftedIEA
findf['intExpDep'] = deYTM(findf["RIAD4170"])/shiftedIEA
findf['intExpTrade'] = deYTM(findf['RIAD4185'])/shiftedIEA
findf['intExpNotes'] = deYTM(findf["RIAD4200"])/shiftedIEA
findf['intIncSec'] = deYTM(findf['RIAD4218'])/shiftedIEA
findf['LoanProvision'] = deYTM(findf['RIAD4230'].add(findf["RIADJJ33"], fill_value = 0).sub(findf["RIADKW02"], fill_value = 0).sub(findf["RIADC901"], fill_value = 0).sub(findf["RIAD4235"], fill_value = 0))/shiftedIEA
findf['othIntInc'] = deYTM(findf["RIAD4518"])/shiftedIEA
findf['othIntExp'] = deYTM(findf["RIADGW44"])/shiftedIEA

c:\Users\bhara\miniconda3\envs\py311\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\bhara\miniconda3\envs\py311\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
findf["liquidityRatio"] = findf[["RCON1350", "RCONB987", "RCONB989", "RCON2146", "RCON3545", "RCON0390", "RCON1754", "RCON1772"]].sum(axis=1, skipna = True)/findf["totAssets"] # Cash missing
findf['assetMatPeriod'] = (findf['RCONA549'] * 1.5).add(findf['RCONA550'] * 7.5, fill_value = 0).add(findf['RCONA551'] * 24, fill_value = 0).add(findf['RCONA552'] * 48, fill_value = 0).add(findf['RCONA553'] * 120, fill_value = 0).add(findf['RCONA554'] * 240, fill_value = 0).add(findf['RCONA555'] * 1.5, fill_value = 0).add(findf['RCONA556'] * 7.5, fill_value = 0).add(findf['RCONA557'] * 24, fill_value = 0).add(findf['RCONA558'] * 48, fill_value = 0).add(findf['RCONA559'] * 120, fill_value = 0).add(findf['RCONA560'] * 240, fill_value = 0).add(findf['RCONA561'] * 18, fill_value = 0).add(findf['RCONA562'] * 60, fill_value = 0).add(findf['RCONA564'] * 1.5, fill_value = 0).add(findf['RCONA565'] * 7.5, fill_value = 0).add(findf['RCONA566'] * 24, fill_value = 0).add(findf['RCONA567'] * 48, fill_value = 0).add(findf['RCONA568'] * 120, fill_value = 0).add(findf['RCONA569'] * 240, fill_value = 0).add(findf['RCONA570'] * 1.5, fill_value = 0).add(findf['RCONA571'] * 7.5, fill_value = 0).add(findf['RCONA572'] * 24, fill_value = 0).add(findf['RCONA573'] * 48, fill_value = 0).add(findf['RCONA574'] * 120, fill_value = 0).add(findf['RCONA575'] * 240, fill_value = 0)/shiftedIEA
findf['assetMatPeriod'] = findf['assetMatPeriod']/12

findf['liabMatPeriod'] = (findf['RCONA579'] * 1.5).add(findf['RCONA580'] * 7.5, fill_value = 0).add(findf['RCONA581'] * 24, fill_value = 0).add(findf['RCONA582'] * 48, fill_value = 0).add(findf['RCONA584'] * 1.5, fill_value = 0).add(findf['RCONA585'] * 7.5, fill_value = 0).add(findf['RCONA586'] * 24, fill_value = 0).add(findf['RCONA587'] * 48, fill_value = 0).add(findf['RCONHK07'] * 1.5, fill_value = 0).add(findf['RCONHK08'] * 7.5, fill_value = 0).add(findf['RCONHK09'] * 24, fill_value = 0).add(findf['RCONHK10'] * 48, fill_value = 0).add(findf['RCONHK12'] * 1.5, fill_value = 0).add(findf['RCONHK13'] * 7.5, fill_value = 0).add(findf['RCONHK14'] * 24, fill_value = 0).add(findf['RCONHK15'] * 48, fill_value = 0).add(findf['RCONF055'] * 6, fill_value = 0).add(findf['RCONF056'] * 24, fill_value = 0).add(findf['RCONF057'] * 48, fill_value = 0).add(findf['RCONF058'] * 60, fill_value = 0).add(findf['RCONF060'] * 6, fill_value = 0).add(findf['RCONF061'] * 24, fill_value = 0).add(findf['RCONF062'] * 48, fill_value = 0).add(findf['RCONF063'] * 60, fill_value = 0)/findf['RCON2948']
findf['liabMatPeriod'] = findf['liabMatPeriod']/12

findf['IRDerivOther'] = findf['RCON8725']
findf['swapsTotal'] = findf['RCON3450']
findf['IRDerivTrade'] = findf['RCONA126']
findf['IRDerivPos'] = findf[["RCON8733", "RCON8741"]].sum(axis=1, skipna=True)
findf['IRDerivNeg'] = findf[["RCON8737", "RCON8745"]].sum(axis=1, skipna=True)
findf['swapsFixed'] = findf['RCONA589']
findf['swapsFloat'] = findf['RCON3450'].sub(findf['RCONA589'])
findf['IRFutFoward'] = findf[["RCON8693", "RCON8697"]].sum(axis=1, skipna=True)
findf['optContSold'] = findf[["RCON8701", "RCON8709"]].sum(axis=1, skipna=True)
findf['optContBought'] = findf[["RCON8705", "RCON8713"]].sum(axis=1, skipna=True)

findf = findf[['IEA', 'matAsset', "qtyAssets", "totLiab", 'totEquity', 'matLiab', 'savingDep', 'demandDep', 'timeDep', "intBearDep", 'totLoans', 'intExpenseRateAnn', 'intIncomeRateAnn', 'niiRateAnn', 'nniRateAnn', 'roaAnn', 'intIncLoans', 'intIncFed', 'intIncOthSec', 'IntIncTrade', 'intIncDepInst', 'intExpDep', 'intExpTrade', 'intExpNotes', 'intIncSec', 'LoanProvision', 'othIntInc', 'othIntExp', 'assetMatPeriod', 'liabMatPeriod', 'othAssets', 'othLiab', 'totAssets', 'IRDerivOther', 'swapsTotal', 'IRDerivTrade', 'swapsFixed', 'swapsFloat', 'deltaLogA', "IRDerivPos", "IRDerivNeg", "IRFutFoward", "optContSold", "optContBought", "totTreasuries", "MADummy", "liquidityRatio"]]
findf['matGap'] = findf['assetMatPeriod'].sub(findf['liabMatPeriod'], fill_value = 0)

print(findf.shape)

(831720, 48)


In [6]:
findf.to_csv("./Data/finalData/findf_RelSeries.csv")

In [10]:
findf[["liquidityRatio"]].to_csv("./Data/bankLiqRatio.csv")